In [1]:
import pandas as pd

In [2]:
import datetime

In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

In [4]:
start_date = '2023-01-01'
end_date = '2023-01-20'
assortment_key = 'MD1'
product_count = 6
shelf_height = 100  # cm
shelf_width = 50    # cm
shelf_depth = 30    # cm (optional)
shelf_volume = shelf_height * shelf_width * shelf_depth  # cm^3

In [5]:
# Read the Excel files
rule_type_1_df = pd.read_excel('bsnrules.xlsx')
result = pd.read_excel('rankingkey.xlsx')

# Remove duplicates from the result DataFrame
result = result.drop_duplicates()

# Create an empty DataFrame to store the final results
final_df = pd.DataFrame()

# Iterate through each rule in rule_type_1_df
for idx, rule in rule_type_1_df.iterrows():
    # Filter the result DataFrame based on the Assortment Key
    matching_products = result[result['Assortment Key'] == rule['Product']]
    
    # For each matching product, create a new rule with the product name
    for _, product in matching_products.iterrows():
        new_rule = rule.copy()
        new_rule['Product'] = product['Product']
        final_df = pd.concat([final_df, new_rule.to_frame().T], ignore_index=True)

# Reset index of the final DataFrame
final_df.reset_index(drop=True, inplace=True)

In [6]:
rule_type_1_df

,Rule ID,Product,Attribute,Operator,Value,RULE_TYPE
0,1,Sunfeast,Sales per Week,>,100,0
1,2,Alphonso Mangoes,Retail Cost per Week,<,3500,0
2,3,Britannia,Sales per Week,>=,150,0
3,4,Minute Maid Orange Juice,Production Cost per Week,<=,3000,0
4,5,REAL Guava Juice,Retail Cost per Week,>,2000,0
5,6,Unibic Foods,Sales per Week,<,200,0
6,7,Maaza,Retail Cost per Week,>=,5000,0
7,8,Tropicano Pomegranate Juice,Production Cost per Week,<=,2000,0
8,9,Parle,Sales per Week,>,120,0
9,10,MD1,Production Cost per Week,<,1700,1


In [7]:
final_df = final_df.drop_duplicates()

In [8]:
rules = pd.concat([rule_type_1_df, final_df], ignore_index=True)

In [9]:
rules

,Rule ID,Product,Attribute,Operator,Value,RULE_TYPE
0,1,Sunfeast,Sales per Week,>,100,0
1,2,Alphonso Mangoes,Retail Cost per Week,<,3500,0
2,3,Britannia,Sales per Week,>=,150,0
3,4,Minute Maid Orange Juice,Production Cost per Week,<=,3000,0
4,5,REAL Guava Juice,Retail Cost per Week,>,2000,0
5,6,Unibic Foods,Sales per Week,<,200,0
6,7,Maaza,Retail Cost per Week,>=,5000,0
7,8,Tropicano Pomegranate Juice,Production Cost per Week,<=,2000,0
8,9,Parle,Sales per Week,>,120,0
9,10,MD1,Production Cost per Week,<,1700,1


In [10]:
df = pd.read_excel('rankingkey.xlsx')

In [11]:
df

,Product,Sales per Week,Retail Cost per Week,Production Cost per Week,Product ID,Assortment Key,Week,Validity
0,BNatural Pomegranate Juice,150,3000,1800,P001,MD1,1,2023-01-01
1,Real Pomegranate Juice,200,4000,2500,P002,MD1,1,2023-01-01
2,Horlicks,100,2500,1400,P003,MD3,1,2023-01-01
3,Alphonso Mangoes,180,5400,3200,P004,MD2,1,2023-01-01
4,Parle,130,2600,1600,P005,MD3,1,2023-01-01
...,...,...,...,...,...,...,...,...
163,Strawberry,210,3500,2050,P017,MD2,8,2023-02-19
164,Cranberry,200,4600,2750,P018,MD2,8,2023-02-19
165,Gala Apple,240,5800,3450,P019,MD2,8,2023-02-19
166,Maaza,230,5500,3150,P020,MD1,8,2023-02-19


In [12]:
mask = (df['Validity'] >= start_date) & (df['Validity'] <= end_date) & (df['Assortment Key'] == assortment_key)

In [13]:
df2 = df[mask]

In [14]:
df2

,Product,Sales per Week,Retail Cost per Week,Production Cost per Week,Product ID,Assortment Key,Week,Validity
0,BNatural Pomegranate Juice,150,3000,1800,P001,MD1,1,2023-01-01
1,Real Pomegranate Juice,200,4000,2500,P002,MD1,1,2023-01-01
6,Tropicano Pomegranate Juice,140,2800,1700,P007,MD1,1,2023-01-01
9,Minute Maid Orange Juice,210,6300,3800,P010,MD1,1,2023-01-01
10,REAL Guava Juice,170,3400,2200,P011,MD1,1,2023-01-01
15,Tropicano Guava Juice,200,6000,3600,P016,MD1,1,2023-01-01
19,Maaza,160,4800,2800,P020,MD1,1,2023-01-01
20,Parle Agros Appy Fizz,200,9000,1000,P021,MD1,1,2023-01-01
21,BNatural Pomegranate Juice,160,3100,1850,P001,MD1,2,2023-01-08
22,Real Pomegranate Juice,210,4100,2550,P002,MD1,2,2023-01-08


In [15]:
df2["(K,D,O)"] = "D"

/var/folders/fs/zvq11zm97ps2908z0gl0wpyr0000gn/T/ipykernel_56952/2536967661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["(K,D,O)"] = "D"


In [16]:
df2

,Product,Sales per Week,Retail Cost per Week,Production Cost per Week,Product ID,Assortment Key,Week,Validity,"(K,D,O)"
0,BNatural Pomegranate Juice,150,3000,1800,P001,MD1,1,2023-01-01,D
1,Real Pomegranate Juice,200,4000,2500,P002,MD1,1,2023-01-01,D
6,Tropicano Pomegranate Juice,140,2800,1700,P007,MD1,1,2023-01-01,D
9,Minute Maid Orange Juice,210,6300,3800,P010,MD1,1,2023-01-01,D
10,REAL Guava Juice,170,3400,2200,P011,MD1,1,2023-01-01,D
15,Tropicano Guava Juice,200,6000,3600,P016,MD1,1,2023-01-01,D
19,Maaza,160,4800,2800,P020,MD1,1,2023-01-01,D
20,Parle Agros Appy Fizz,200,9000,1000,P021,MD1,1,2023-01-01,D
21,BNatural Pomegranate Juice,160,3100,1850,P001,MD1,2,2023-01-08,D
22,Real Pomegranate Juice,210,4100,2550,P002,MD1,2,2023-01-08,D


In [17]:
df2.shape

(24, 9)

In [18]:
df2["Product"].unique()

array(['BNatural Pomegranate Juice', 'Real Pomegranate Juice',
       'Tropicano Pomegranate Juice', 'Minute Maid Orange Juice',
       'REAL Guava Juice', 'Tropicano Guava Juice', 'Maaza ',
       'Parle Agros Appy Fizz'], dtype=object)

In [19]:
agg_functions = {'Sales per Week': 'mean',
                 'Retail Cost per Week': 'mean',
                 'Production Cost per Week': 'mean',
                 'Product': 'first',
                 'Assortment Key': 'first',
                 'Week': 'first',
                 'Validity': 'first',
                 '(K,D,O)': 'first'}

In [20]:
df2_agg = df2.groupby('Product').agg(agg_functions)

In [21]:
df2_agg

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)"
Product,,,,,,,,
BNatural Pomegranate Juice,160.0,3100.0,1850.0,BNatural Pomegranate Juice,MD1,1,2023-01-01,D
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,D
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,D
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,D
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,D
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,D
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,D
Tropicano Pomegranate Juice,150.0,2900.0,1750.0,Tropicano Pomegranate Juice,MD1,1,2023-01-01,D


In [22]:
df2_agg

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)"
Product,,,,,,,,
BNatural Pomegranate Juice,160.0,3100.0,1850.0,BNatural Pomegranate Juice,MD1,1,2023-01-01,D
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,D
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,D
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,D
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,D
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,D
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,D
Tropicano Pomegranate Juice,150.0,2900.0,1750.0,Tropicano Pomegranate Juice,MD1,1,2023-01-01,D


In [23]:
for index, row in rules.iterrows():
    product = row['Product']
    attribute = row['Attribute']
    operator = row['Operator']
    value = row['Value']
 
 
    if product in df['Product'].values:
 
        condition_met = False
        product_row = df[df['Product'] == product].iloc[0]
        print(product_row)
        print("--------------")
        print(product_row[attribute],operator,  value)
 
 
        if operator == '>':
            condition_met = product_row[attribute] > value
            print(condition_met)
        elif operator == '<':
            condition_met = product_row[attribute] < value
            print(condition_met)
        elif operator == '>=':
            condition_met = product_row[attribute] >= value
            print(condition_met)
        elif operator == '<=':
            condition_met = product_row[attribute] <= value
            print(condition_met)
        elif operator == '==':
            condition_met = product_row[attribute] == value
            print(condition_met)
 
        if condition_met:
            print(f"{product}: BR satisfied")
            print(product)
            print("=======================")
        else:
            print(f"{product}: BR unsatisfied")
            print(product)
            df.loc[df["Product"] == product, "(K,D,O)"] = "O"
            print("=======================")
 
    else:
        print(f"{product}: Product not found in df DataFrame")

Product                                Sunfeast
Sales per Week                              160
Retail Cost per Week                       4800
Production Cost per Week                   2900
Product ID                                 P009
Assortment Key                              MD3
Week                                          1
Validity                    2023-01-01 00:00:00
Name: 8, dtype: object
--------------
160     > 100
Sunfeast: BR unsatisfied
Sunfeast
Product                        Alphonso Mangoes
Sales per Week                              180
Retail Cost per Week                       5400
Production Cost per Week                   3200
Product ID                                 P004
Assortment Key                              MD2
Week                                          1
Validity                    2023-01-01 00:00:00
(K,D,O)                                       n
Name: 3, dtype: object
--------------
5400 < 3500
False
Alphonso Mangoes: BR unsatisfied
Alphonso 

In [24]:
df2_agg

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)"
Product,,,,,,,,
BNatural Pomegranate Juice,160.0,3100.0,1850.0,BNatural Pomegranate Juice,MD1,1,2023-01-01,D
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,D
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,D
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,D
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,D
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,D
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,D
Tropicano Pomegranate Juice,150.0,2900.0,1750.0,Tropicano Pomegranate Juice,MD1,1,2023-01-01,D


In [25]:
# normalizing

df2_agg['Normalized Sales'] = (df2_agg['Sales per Week'] - df2_agg['Sales per Week'].min()) / (df2_agg['Sales per Week'].max() - df2_agg['Sales per Week'].min())
df2_agg['Profit'] = df2_agg['Retail Cost per Week'] - df2_agg['Production Cost per Week']
df2_agg['Normalized Profit'] = (df2_agg['Profit'] - df2_agg['Profit'].min()) / (df2_agg['Profit'].max() - df2_agg['Profit'].min())

#ranking key here
df2_agg['Ranking Key'] = 0.5 * df2_agg['Normalized Sales'] + 0.5 * df2_agg['Normalized Profit']


def get_top_products(product_count):
   
    sorted_df = df2_agg.sort_values(by='Ranking Key', ascending=False)
   
    top_products = sorted_df.head(product_count)
    return top_products

#product count im taking as a number for now 



top_products = get_top_products(product_count)

print(top_products[['Product', 'Sales per Week', 'Retail Cost per Week', 'Production Cost per Week', 'Ranking Key']])


                                           Product  Sales per Week  \
Product                                                              
Parle Agros Appy Fizz        Parle Agros Appy Fizz           210.0   
Minute Maid Orange Juice  Minute Maid Orange Juice           220.0   
Tropicano Guava Juice        Tropicano Guava Juice           210.0   
Real Pomegranate Juice      Real Pomegranate Juice           210.0   
REAL Guava Juice                  REAL Guava Juice           180.0   
Maaza                                       Maaza            170.0   

                          Retail Cost per Week  Production Cost per Week  \
Product                                                                    
Parle Agros Appy Fizz                   9100.0                    1050.0   
Minute Maid Orange Juice                6400.0                    3850.0   
Tropicano Guava Juice                   6100.0                    3650.0   
Real Pomegranate Juice                  4100.0             

In [26]:
top_products

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)",Normalized Sales,Profit,Normalized Profit,Ranking Key
Product,,,,,,,,,,,,
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,D,0.857143,8050.0,1.000000,0.928571
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,D,1.000000,2550.0,0.202899,0.601449
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,D,0.857143,2450.0,0.188406,0.522774
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,D,0.857143,1550.0,0.057971,0.457557
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,D,0.428571,1250.0,0.014493,0.221532
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,D,0.285714,2050.0,0.130435,0.208075


In [27]:
mask = df2_agg['Product'].isin(top_products['Product'])

In [28]:
df2_agg.loc[mask, "(K,D,O)"] = df2_agg.loc[mask, "(K,D,O)"].apply(lambda x: "K")

In [29]:
df2_agg.head(30)

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)",Normalized Sales,Profit,Normalized Profit,Ranking Key
Product,,,,,,,,,,,,
BNatural Pomegranate Juice,160.0,3100.0,1850.0,BNatural Pomegranate Juice,MD1,1,2023-01-01,D,0.142857,1250.0,0.014493,0.078675
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,K,0.285714,2050.0,0.130435,0.208075
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,K,1.000000,2550.0,0.202899,0.601449
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,K,0.857143,8050.0,1.000000,0.928571
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,K,0.428571,1250.0,0.014493,0.221532
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,K,0.857143,1550.0,0.057971,0.457557
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,K,0.857143,2450.0,0.188406,0.522774
Tropicano Pomegranate Juice,150.0,2900.0,1750.0,Tropicano Pomegranate Juice,MD1,1,2023-01-01,D,0.000000,1150.0,0.000000,0.000000


In [30]:
df = df2_agg

In [31]:
df

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)",Normalized Sales,Profit,Normalized Profit,Ranking Key
Product,,,,,,,,,,,,
BNatural Pomegranate Juice,160.0,3100.0,1850.0,BNatural Pomegranate Juice,MD1,1,2023-01-01,D,0.142857,1250.0,0.014493,0.078675
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,K,0.285714,2050.0,0.130435,0.208075
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,K,1.000000,2550.0,0.202899,0.601449
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,K,0.857143,8050.0,1.000000,0.928571
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,K,0.428571,1250.0,0.014493,0.221532
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,K,0.857143,1550.0,0.057971,0.457557
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,K,0.857143,2450.0,0.188406,0.522774
Tropicano Pomegranate Juice,150.0,2900.0,1750.0,Tropicano Pomegranate Juice,MD1,1,2023-01-01,D,0.000000,1150.0,0.000000,0.000000


In [32]:
# df.rename(columns = {'Product':'Product Name'}, inplace = True)

In [33]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [34]:

excel_df = pd.read_excel("ProductDimensions.xlsx")
 
filtered_df = df[df['(K,D,O)'] == 'K']

 
 

In [35]:
filtered_df

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)",Normalized Sales,Profit,Normalized Profit,Ranking Key
Product,,,,,,,,,,,,
Maaza,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,K,0.285714,2050.0,0.130435,0.208075
Minute Maid Orange Juice,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,K,1.000000,2550.0,0.202899,0.601449
Parle Agros Appy Fizz,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,K,0.857143,8050.0,1.000000,0.928571
REAL Guava Juice,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,K,0.428571,1250.0,0.014493,0.221532
Real Pomegranate Juice,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,K,0.857143,1550.0,0.057971,0.457557
Tropicano Guava Juice,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,K,0.857143,2450.0,0.188406,0.522774


In [36]:
excel_df

,Product ID,Product Name,Height (cm),Width (cm),Depth (cm)
0,P001,BNatural Pomegranate Juice,20.3,7.6,7.6
1,P002,Real Pomegranate Juice,20.3,7.2,7.6
2,P003,Horlicks,20.3,10.2,10.2
3,P004,Alphonso Mangoes,10.2,7.4,7.6
4,P005,Parle,15.2,7.8,5.1
5,P006,Britannia,15.2,7.9,5.1
6,P007,Tropicano Pomegranate Juice,20.3,7.3,7.6
7,P008,Totapuri Mangoes,12.7,7.2,7.6
8,P009,Sunfeast,15.2,7.6,5.1
9,P010,Minute Maid Orange Juice,20.3,7.7,7.6


In [37]:

# Assuming excel_df is loaded from Excel
excel_df = pd.read_excel("productDimensions.xlsx")  # Adjust the filename as needed
filtered_df = filtered_df.reset_index(drop=True)
# Merge DataFrames on the 'Product' column
merged_df = pd.merge(filtered_df, excel_df, left_on='Product', right_on='Product Name')

In [38]:
merged_df

,Sales per Week,Retail Cost per Week,Production Cost per Week,Product,Assortment Key,Week,Validity,"(K,D,O)",Normalized Sales,Profit,Normalized Profit,Ranking Key,Product ID,Product Name,Height (cm),Width (cm),Depth (cm)
0,170.0,4900.0,2850.0,Maaza,MD1,1,2023-01-01,K,0.285714,2050.0,0.130435,0.208075,P020,Maaza,20.3,7.4,7.6
1,220.0,6400.0,3850.0,Minute Maid Orange Juice,MD1,1,2023-01-01,K,1.000000,2550.0,0.202899,0.601449,P010,Minute Maid Orange Juice,20.3,7.7,7.6
2,210.0,9100.0,1050.0,Parle Agros Appy Fizz,MD1,1,2023-01-01,K,0.857143,8050.0,1.000000,0.928571,P021,Parle Agros Appy Fizz,20.3,7.6,7.6
3,180.0,3500.0,2250.0,REAL Guava Juice,MD1,1,2023-01-01,K,0.428571,1250.0,0.014493,0.221532,P011,REAL Guava Juice,20.3,7.1,7.6
4,210.0,4100.0,2550.0,Real Pomegranate Juice,MD1,1,2023-01-01,K,0.857143,1550.0,0.057971,0.457557,P002,Real Pomegranate Juice,20.3,7.2,7.6
5,210.0,6100.0,3650.0,Tropicano Guava Juice,MD1,1,2023-01-01,K,0.857143,2450.0,0.188406,0.522774,P016,Tropicano Guava Juice,20.3,7.6,7.6


In [39]:
profits = merged_df['Profit'].values
heights = merged_df['Height (cm)'].values
widths = merged_df['Width (cm)'].values
depths = merged_df['Depth (cm)'].values
products = merged_df['Product'].values

In [40]:
volumes = heights * widths * depths

In [41]:
prob = LpProblem(
"Maximize_Profit"
, LpMaximize)

In [42]:
units = LpVariable.dicts(
"Units"
, products, lowBound=0, cat=
'Integer'
)

In [43]:
prob += lpSum([units[product] * profits[i] for i, product in enumerate(products)])

In [44]:
prob += lpSum([units[product] * volumes[i] for i, product in enumerate(products)]) <= shelf_volume
 
# Add a constraint to ensure that at least 1 unit of each product is included
for i, product in enumerate(products):
    prob += units[product] >= 1

In [45]:
# Add constraints to balance the number of units across products
# For example, ensure no single product has more than 10 times the units of another product
for i in range(len(products)):
    for j in range(len(products)):
        prob += units[products[i]] <= 10 * units[products[j]]

In [46]:
# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fs/zvq11zm97ps2908z0gl0wpyr0000gn/T/8d569c6cca70488cb183d2c3e5821bcc-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fs/zvq11zm97ps2908z0gl0wpyr0000gn/T/8d569c6cca70488cb183d2c3e5821bcc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 145 RHS
At line 189 BOUNDS
At line 196 ENDATA
Problem MODEL has 43 rows, 6 columns and 78 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 777376 - 0.00 seconds
Cgl0004I processed model has 31 rows, 6 columns (6 integer (0 of which binary)) and 66 elements
Cutoff increment increased from 1e-05 to 49.9999
Cbc0012I Integer solution of -743200 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0012I Int

1

In [47]:
if LpStatus[prob.status] != 'Optimal':
    print(f"Optimization problem did not find an optimal solution. Status: {LpStatus[prob.status]}")
 
# Collect the results
results = []
 
for i, product in enumerate(products):
    results.append({
        'Product': product,
        'Units': units[product].varValue,
        'Profit': units[product].varValue * profits[i]
    })
 

In [48]:
results_df = pd.DataFrame(results)

In [49]:
results_df

,Product,Units,Profit
0,Maaza,9.0,18450.0
1,Minute Maid Orange Juice,9.0,22950.0
2,Parle Agros Appy Fizz,84.0,676200.0
3,REAL Guava Juice,9.0,11250.0
4,Real Pomegranate Juice,9.0,13950.0
5,Tropicano Guava Juice,9.0,22050.0


In [50]:
results_df = results_df[results_df[
'Units'
] > 0]

In [51]:
total_used_volume = sum(results_df['Units'] * volumes)
print(f"Total used volume: {total_used_volume} cm^3 out of {shelf_volume} cm^3")
 

Total used volume: 149867.592 cm^3 out of 150000 cm^3
